<a href="https://colab.research.google.com/github/bishair/Real-time-model/blob/main/TestModelOnNewData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import necessary libraries for the model and plotting
import numpy as np
import pandas as pd
from google.colab import files
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
from math import sqrt
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [21]:
# Upload excel file
uploaded = files.upload()

Saving testdataCSV.csv to testdataCSV (1).csv


In [4]:
#Load data into pandas dataframe
df = pd.read_csv('processedCSV.csv', sep=';', decimal=',')

In [5]:
df.head()

,DateGW,G10,RiverLevel
0,2015-01-30 13:00:00,8.555,260
1,2015-01-30 14:00:00,8.557,260
2,2015-01-30 15:00:00,8.559,260
3,2015-01-30 16:00:00,8.562,260
4,2015-01-30 17:00:00,8.563,260


In [6]:
# Convert 'DateGW' to datetime and set as index
df['DateGW'] = pd.to_datetime(df['DateGW'])
df.set_index('DateGW', inplace=True)

In [7]:
# Columns to scale
data = df[['G10', 'RiverLevel']].values

In [8]:
# Normalize the features
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

In [9]:
# Prepare the data for LSTM
def create_dataset(data, sequence_length):
    X, y = [], []
    for i in range(sequence_length, len(data)):
        X.append(data[i-sequence_length:i])
        y.append(data[i, 0])  # 0 for G10 groundwater level at current time
    return np.array(X), np.array(y)


In [10]:
sequence_length = 24  # Look 24 hours back to predict the next hour
X, y = create_dataset(scaled_data, sequence_length)

In [11]:
# Reshape features for LSTM Layer [samples, time steps, features]
X = np.reshape(X, (X.shape[0], sequence_length, X.shape[2]))

In [12]:
# Split the data into training and testing sets
train_size = int(len(X) * 0.7)
test_size = len(X) - train_size
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

In [13]:
# Build the LSTM model
model = Sequential()
model.add(LSTM(50, return_sequences=False, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(1))  # predicting one value

In [14]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

In [15]:
# Train the model
history = model.fit(X_train, y_train, epochs=100, batch_size=64, verbose=1, validation_split=0.1)

Epoch 1/100
173/173 [==============================] - 5s 15ms/step - loss: 0.0165 - val_loss: 5.9650e-05
Epoch 2/100
173/173 [==============================] - 3s 19ms/step - loss: 2.6293e-05 - val_loss: 4.4442e-05
Epoch 3/100
173/173 [==============================] - 2s 13ms/step - loss: 2.4082e-05 - val_loss: 3.8485e-05
Epoch 4/100
173/173 [==============================] - 2s 12ms/step - loss: 2.2007e-05 - val_loss: 4.4503e-05
Epoch 5/100
173/173 [==============================] - 2s 12ms/step - loss: 2.0046e-05 - val_loss: 3.3273e-05
Epoch 6/100
173/173 [==============================] - 2s 12ms/step - loss: 1.8258e-05 - val_loss: 3.6752e-05
Epoch 7/100
173/173 [==============================] - 2s 13ms/step - loss: 1.6226e-05 - val_loss: 3.1544e-05
Epoch 8/100
173/173 [==============================] - 3s 18ms/step - loss: 1.4625e-05 - val_loss: 2.6543e-05
Epoch 9/100
173/173 [==============================] - 2s 12ms/step - loss: 1.3291e-05 - val_loss: 2.5516e-05
Epoch 10/100
1

In [16]:
model.save('my_lstm_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [17]:
# Predictions
predicted_groundwater_level = model.predict(X_test)

165/165 [==============================] - 1s 4ms/step


In [18]:
# Inverse transform the normalized values to the original scale
predicted_groundwater_level = scaler.inverse_transform(np.hstack((predicted_groundwater_level, np.zeros((predicted_groundwater_level.shape[0], scaled_data.shape[1]-1)))))
y_test = scaler.inverse_transform(np.hstack((y_test.reshape(-1, 1), np.zeros((y_test.shape[0], scaled_data.shape[1]-1)))))[:, 0]

In [19]:
# Calculate RMSE
rmse = sqrt(mean_squared_error(y_test, predicted_groundwater_level[:, 0]))
print('Test RMSE: %.3f' % rmse)

Test RMSE: 0.003


#Evaluate performanc of trained model on unseen data

In [30]:
# Import necessary libraries for the model and plotting
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import load_model

# Load the saved LSTM model
model = load_model('my_lstm_model.h5')  # Replace with the actual path to your saved model

# Load the new data from the test CSV file
test_df = pd.read_csv('testdataCSV.csv', sep=';', decimal=',')  # Replace with the path to your test data CSV

# Preprocess the test data similarly to the training data

# Convert 'DateGW' to datetime and set as index
test_df['DateGW'] = pd.to_datetime(test_df['DateGW'])
test_df.set_index('DateGW', inplace=True)

# Columns to scale (make sure it matches the columns used during training)
test_data = test_df[['G10', 'RiverLevel']].values

# Normalize the features using the same scaler used during training
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(test_data)  # Fit the scaler to the test data (not scaled data)

# Define the sequence length used during training
sequence_length = 24

# Initialize an empty array to store the predictions
all_predictions = []

# Iterate over the new data in suitable windows
for i in range(len(test_data) - sequence_length + 1):
    # Extract a window of data
    window_data = test_data[i:i+sequence_length]

    # Scale the window data
    window_data_scaled = scaler.transform(window_data)

    # Reshape the window data for LSTM Layer [samples, time steps, features]
    window_data_reshaped = np.reshape(window_data_scaled, (1, sequence_length, window_data_scaled.shape[1]))

    # Make predictions for the window
    window_predictions = model.predict(window_data_reshaped)

    # Ensure that 'window_predictions' has the expected shape for inverse transformation
    window_predictions = window_predictions.reshape(1, -1)  # Reshape to (1, num_features)

    # Inverse transform the predictions
    window_predictions = scaler.inverse_transform(window_predictions)

    # Append the predictions for this window to the results
    all_predictions.extend(window_predictions[0])  # Extract values from the array


1/1 [==============================] - 0s 413ms/step


ValueError: ignored